## Imports

In [1]:
!pip3 install python-binance

     |████████████████████████████████| 778kB 16.0MB/s 
     |████████████████████████████████| 3.1MB 45.4MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 358kB 50.7MB/s 
     |████████████████████████████████| 2.3MB 53.0MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 174kB 38.3MB/s 
     |████████████████████████████████| 645kB 47.0MB/s 


In [2]:

%tensorflow_version 2.x
import trading_helper as th
import numpy as np
import pandas as pd
import time
import os
import tensorflow
import tensorflow as tf
import json
from datetime import datetime, timedelta, timezone

TensorFlow 2.x selected.


## Trading strategy
Here is a simple trading strategy meat to illustrate the bare minimum.

See the (Python Binance API)[https://python-binance.readthedocs.io/en/latest/] for more functionality.

Group4 "Become A Millionaire" Strategy

In [0]:
def strategy(model, keys_fname, log_fname, win_sz, trading_interval='12h'):
    """
    Arguments
    --------- 
    model            : Some trained forecasting model
    keys_fname       : File name to the keys, see below
    log_fname        : You will need to log the trades you submit to Binance. 
                       The file name should be on the format "tradingbot_group<number>_<date_time>.log". 
                       E.g. "tradingbot_group1_2019-11-13_00:23:23.log"
    win_sz           : The window size that you specified when training the model, 
                       i.e. how many time steps of historical data the model needs for a forecast.
    trading_interval : How often the trading bot will query data from Binance, 
                       make a new forecast and decide whether to trade. 
                       E.g. '1min', '5min', '15min', '30min', '1h', '2h', '4h', '6h', '12h', '1d' or '1w'.
    """
    
    # use the keys stored in KEYS_FNAME to initialize the Binance client
    client = th.init_client(keys_fname)
    
    # convert the trading_interval string to seconds (int)
    interval_s = th.interval_string_to_seconds(trading_interval)
    
    # initialize the log file
    th.log(log_fname, new_log=True)
        
    # initialize trading with historical data
    t = datetime.utcnow()
    # timedelta(weeks=2) -> 2 weeks back in time
    t = th.UTC('time') - timedelta(weeks=7)
    # get data to begin forecasting from
    data = th.get_klines([t.year, t.month, t.day, t.hour, t.minute, t.second], interval=trading_interval)
    
    # forecast 
    x = data['returns'][-win_sz:].to_numpy().reshape((win_sz, 1))
    forecast = model.predict(x)
    
    while True:
    
        try:

            if forecast[-1] > 0.0008:  # forcasted price increase -> buy

                # decide on how to set the entry price
                entry_price = data['high'][-1] 
                risk = 0.7  # all in, max risk
                # get the current balance
                balance = client.get_asset_balance(asset='USDT')
                entry_capital = np.float64(balance['free']) * risk # balance['free'] is a str -> convert to float
                
                if entry_capital > 0:
                    order = th.limit_buy(client, 'BTCUSDT', entry_price, quantity=entry_capital)
                    
                    th.log(log_fname, order_type='NEW_BUY', quantity=order['origQty'],
                        price=entry_price, time=th.UTC('iso'))

            elif forecast[-1] < 0:  # forcasted price decrease
                exit_price = data['low'][-1] 
                balance = client.get_asset_balance(asset='BTC')
                risk = 0.6  # all in, max risk
                exit_capital = np.float64(balance['free']) * risk # balance['free'] is a str -> convert to float

                if entry_capital > 0:
                    order = th.limit_sell(client, 'BTCUSDT', exit_price, quantity=exit_capital)
                    th.log(log_fname, order_type='NEW_SELL', quantity=order['origQty'],
                        price=exit_price, time=th.UTC('iso'))
                    
            # pause/sleep for the trading_interval
            time.sleep(interval_s)
            # get last time from data
            t = data.index[-1]
            # update data 
            tmp = h.get_klines([t.year, t.month, t.day, t.hour, t.minute, t.second], interval=trading_interval)
            data.append(tmp)
            forecast = model.predict(data['returns'][-win_sz:])
                
        except ValueError:
            pass  # Handles lookback errors in beginning of dataset

## Live trading on Binance

In [6]:
# load pre-trained model

#model_name = 'BidirLSTM_96Gold2_epochs300.h5'
#model_dir = os.path.join('models', model_name)
#fname_model = 'BidirLSTM_96Gold2_epochs300.h5' % (model_dir, model_name, epochs)


model = tf.keras.models.load_model('BidirLSTM_96Gold2_epochs300.h5')
keys_fname = 'Group4_key.json' # set this file name to your file name
log_fname = 'tradingbot_group4_%s.log' % (th.UTC('iso')[:-10]) # 
win_sz = 50  # example
trading_interval = '12h' # example
strategy(model, keys_fname, log_fname, win_sz=50, trading_interval=trading_interval)

	 2019-11-20 23:59


KeyboardInterrupt: ignored